<a href="https://colab.research.google.com/github/julianVelandia/FinetuningLLMGradeWorksUNALDatasetInstruct/blob/master/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/user-attachments/assets/a200c62f-b3d0-4c44-a956-f19f268fc543" alt="image" width="400"/>


In [ ]:
!pip install datasets
!pip install peft transformers torch


In [ ]:
import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model
import torch

In [ ]:
DATASET_NAME = "JulianVelandia/unal-repository-dataset-train-instruct"
MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
OUTPUT_DIR = "JulianVelandia/Llama-3-8B-unal-instruct-ft"
LORA_RANK = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.1
HUGGINGFACE_TOKEN = ":)"

In [ ]:
dataset = load_dataset(DATASET_NAME, split="train")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
for idx, row in enumerate(dataset):
    print(f"Fila {idx + 1}: Tipo - {type(row)}, Contenido - {repr(row)}")
    if idx == 4:
        break


dataset_info = {
    "Tipo": type(dataset),
    "Tamaño": len(dataset),
    "Características": dataset.column_names,
}
print(dataset_info)

Fila 1: Tipo - <class 'dict'>, Contenido - {'prompt': '¿Qué tipo de diseño de estudio se está utilizando para investigar la enfermedad nosocomial NAVM en el hospital?', 'completion': 'Diseño anidado en cohorte.', 'fragment': 'con diagnostico confirmado de NAVM. Este comité multidisciplinario, se encuentra funcionando desde el 15 de abril del 2016 y usa criterios internacionalmente aceptados para el diagnóstico de NAVM, además la NAVM como enfermedad nosocomial hace parte de los reportes oficiales de vigilancia epidemiológica que el hospital debe informar al Instituto Nacional de Salud (INS) y que se llenan de forma rutinaria en fichas epidemiológicas según lo establecido por el Ministerio de la Protección social en Colombia desde el año 2016. Todos los pacientes con diagnóstico de NAVM serán incluidos como casos verificando la presencia de criterios de inclusión y exclusión, los controles se tomarán de forma aleatoria (4 por cada caso) al momento de la aparición temporal del caso (dise

In [ ]:
!pip install -U bitsandbytes accelerate
!nvidia-smi
!nvcc --version


Tue Jan 14 02:26:33 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from bitsandbytes import __version__
print("Versión de bitsandbytes instalada:", __version__)

bnb_config = BitsAndBytesConfig(load_in_8bit=True)
token = HUGGINGFACE_TOKEN

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    token=token
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=token)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.resize_token_embeddings(len(tokenizer))

lora_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

Versión de bitsandbytes instalada: 0.45.0


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples["prompt"], truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(examples["completion"], truncation=True, padding="max_length", max_length=512)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["fragment"])

print("\nDataset tokenizado:")
print(tokenized_dataset)

Map:   0%|          | 0/12525 [00:00<?, ? examples/s]


Dataset tokenizado:
Dataset({
    features: ['prompt', 'completion', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 12525
})


In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir=f"{OUTPUT_DIR}/logs",
    fp16=True,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-367ece0be2ae>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print("Entrenando")
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Entrenando


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,1.145100


In [ ]:
print('subir')
model.push_to_hub(OUTPUT_DIR, token=token)
tokenizer.push_to_hub(OUTPUT_DIR, token=token)

In [ ]:
model.eval()
input_text = "¿Cómo se determina la vida útil de un proyecto según la norma ISO 15686?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
print("Texto generado:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
print('subir')
model.push_to_hub(OUTPUT_DIR, token=token)
tokenizer.push_to_hub(OUTPUT_DIR, token=token)